In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
# Install necessary libraries
# !pip install transformers spacy gensim nltk google-cloud-language
# !pip install --upgrade google-cloud-language
# !pip install python-docx PyPDF2
# !python -m spacy download en_core_web_sm

# Import required libraries
import os
from google.colab import files
from transformers import pipeline
import spacy
from gensim import corpora
from gensim.models import LdaModel
from nltk.tokenize import word_tokenize
from google.cloud import language_v1
from nltk.corpus import stopwords
import nltk

# Ensure required NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

# Function to dynamically upload and extract text
def upload_and_extract_text():
    print("Upload your document (PDF, DOCX, or TXT):")
    uploaded = files.upload()
    file_path = list(uploaded.keys())[0]

    if file_path.endswith(".txt"):
        with open(file_path, "r") as file:
            text = file.read()
    elif file_path.endswith(".pdf"):
        from PyPDF2 import PdfReader
        reader = PdfReader(file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    elif file_path.endswith(".docx"):
        from docx import Document
        doc = Document(file_path)
        text = "\n".join([p.text for p in doc.paragraphs])
    else:
        raise ValueError("Unsupported file type. Please upload a PDF, DOCX, or TXT file.")

    return text

# Function to summarize text (handles long text via chunking)
def summarize_text(text, max_token_limit=1024):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    chunks = []

    # Split text into manageable chunks
    while len(text) > max_token_limit:
        split_point = text.rfind(" ", 0, max_token_limit)
        chunks.append(text[:split_point])
        text = text[split_point:]
    chunks.append(text)  # Append the final chunk

    # Summarize each chunk
    summaries = []
    for chunk in chunks:
        # Adjust max_length based on input length
        input_length = len(chunk.split())
        max_length = min(input_length // 2, 150)
        summary = summarizer(chunk, max_length=max_length, min_length=50, do_sample=False)
        summaries.append(summary[0]['summary_text'])

    # Combine all summaries
    return " ".join(summaries)

# Function for Named Entity Recognition (NER) using spaCy
def extract_entities(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Function for topic modeling using Gensim
def topic_modeling(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalnum() and word not in stopwords.words("english")]
    dictionary = corpora.Dictionary([tokens])
    corpus = [dictionary.doc2bow(tokens)]
    lda_model = LdaModel(corpus, num_topics=1, id2word=dictionary, passes=10)
    topics = lda_model.print_topics(num_words=5)
    return topics

# Function for sentiment analysis using Google Cloud Natural Language API
def analyze_sentiment(text):
    client = language_v1.LanguageServiceClient()
    document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)
    sentiment = client.analyze_sentiment(request={"document": document}).document_sentiment
    return {"score": sentiment.score, "magnitude": sentiment.magnitude}

# Main document review function
def process_document():
    try:
        text = upload_and_extract_text()

        print("\n--- Summarized Text ---")
        summary = summarize_text(text)
        print(summary)

        # print("\n--- Named Entities ---")
        # entities = extract_entities(text)
        # for entity in entities:
        #     print(f"Entity: {entity[0]}, Type: {entity[1]}")

        # print("\n--- Topics ---")
        # topics = topic_modeling(text)
        # for topic in topics:
        #     print(f"Topic: {topic}")


    except Exception as e:
        print(f"Error: {e}")

# Run the document review process
process_document()


Upload your document (PDF, DOCX, or TXT):


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Saving OEM-CORES-Thermal-Stereo-Vision.pdf to OEM-CORES-Thermal-Stereo-Vision (4).pdf

--- Summarized Text ---


Your min_length=50 must be inferior than your max_length=46.


Autonomous vehicles must leverage a variety of sensor types fused together. These include visible cameras, radar, LIDAR, sonar and thermal imaging. Of these sensors, thermal imaging is particularly useful for helping a vehicle “see” in challenging weather and lighting conditions. The ability to make these vehicles truly autonomous anytime, anywhere, remains a technical challenge. Autonomous vehicles of the future must be able to determine, not simply guess, which areas are appropriate for moving vehicles, day or night. For instance, unmapped areas or places that don’t have lanes or include uncommon infrastructure and environmental features, still pose Thermal cameras detect and measure an entirely different wavelength of energy compared to other sensors, known as long-wave infrared (LWIR) radiation, or heat energy. This part of the electromagnetic spectrum radiates, absorbs, or is reflected by everything on Earth. Therefore, thermal cameras detect heat equally well in daylight Thermal 